## 데이터 수집 & 전처리

In [49]:
#API 호출 및 데이터 품질 검사
import re
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

## API로 데이터 불러와서 활용 (가능하면)
- https://www.safetydata.go.kr/disaster-data/view?dataSn=3287#none 에서 파일 불러오고, 활용가능하다 판단되면 활용해보기.
- 지금은 PDF 파일만 수집 활용중

In [22]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

# PDF 파일들이 저장된 디렉토리
directory_path = "SourceCode/RAG/"

# PDF 파일 로드
all_docs = []  # 모든 문서를 저장할 리스트

# 디렉토리에서 모든 파일을 순회
for file_name in os.listdir(directory_path):
    if file_name.endswith(".pdf"):  # PDF 파일만 처리
        file_path = os.path.join(directory_path, file_name)  # 파일 경로 생성
        loader = PyPDFLoader(file_path=file_path)  # PyPDFLoader 생성
        docs = loader.load()  # 문서 로드
        all_docs.extend(docs)  # 전체 문서 리스트에 추가

# print(f"총 {len(all_docs)}개의 페이지가 로드되었습니다.")

In [23]:
def preprocess_text(text):
    '''전처리 함수
    설명 : 추가할 부분 추가하기'''
    #  처음에 숫자가 나오면 제거
    text = re.sub(r'^\s*\d+\s*', '', text)   
    
    #  "비상시 국민행동요령 알아야 안전하다"로 시작하면 제거
    if text.startswith('비상시 국민행동요령 알아야 안전하다'):
        text = text[len('비상시 국민행동요령 알아야 안전하다'):].lstrip()
    
    #  특정 패턴이 시작 부분에 있으면 제거 (패턴 1)
    pattern1 = r'^(\s*만화로 보는 비상시\s*국민행동요령\s*화생방 피해대비\s*행동요령\s*인명시설 피해시\s*행동요령\s*비상대비물자\s*준비 및 사용요령\s*비상사태시\s*행동요령\s*)'
    text = re.sub(pattern1, '', text)
    
    #  특정 패턴이 시작 부분에 있으면 제거 (패턴 2)
    pattern2 = r'^(\s*온 가족이 함께\s*안전하게\s*화생방 피해대비\s*행동요령\s*인명시설 피해시\s*행동요령\s*비상대비물자\s*준비 및 사용요령\s*비상사태시\s*행동요령\s*화생방경보 발령시\s*국민행동요령\s*핵 경보 발령시\s*국민행동요령\s*)'
    text = re.sub(pattern2, '', text)
    
    #  불필요한 줄 바꿈과 공백 정리
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)

    #  한글,. - % / ()를 제외한 모든 특수문자 제거
    text = re.sub(r'[^a-zA-Z0-9\s%.-/()\uAC00-\uD7A3]+', '', text)

    #  문자열 양쪽 공백 제거
    text = text.strip()
    
    return text


In [24]:
# 각 문서의 page_content 전처리
for doc in all_docs:
    doc.page_content = preprocess_text(doc.page_content)

In [25]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(all_docs)

# 청크 확인 디버그 코드
for idx, chunk in enumerate(splits):
    print(f"Chunk {idx + 1}:")
    print("-" * 20)
    print(chunk.page_content)
    print("\n" + "=" * 40 + "\n")

Chunk 1:
--------------------
RUN HIDE TELL www.nctc.go.kr


Chunk 2:
--------------------
11 12 13 14 15 16 18 19 20 21 06 07 우편물택배 테러 화학생물 테러 해외여행 중 테러 억류납치 테러 총기난사 테러 차량돌진 테러 폭발물 테러 항공기 피랍 드론 테러 다중이용시설 테러 방사능 테러 테러는 우리나라와 상관없지 않나요 세계를 뒤흔든 테러 사건 테러경보 단계 테러의심피해상황 즉시 신고 테러 신고요령 다중이용시설 신변안전 해외여행 위험국가 방문 자제 여행경보 단계 해외여행 중 피해예방 수칙 08 테러 유형별 행동요령 06 꾸준히 증가하는 테러 사건 22 테러 피해 예방 일반 수칙 34 대테러센터 소개 주요 임무 홈페이지와 유튜브 채널 테러방지법 소개 23 24 25 26 27 28 30 34 35 36 CONTENTS 테러대비 행동요령 (www.nctc.go.kr)  0302  테러대비 행동요령 (www.nctc.go.kr)


Chunk 3:
--------------------
112      04  테러대비 행동요령 (www.nctc.go.kr)


Chunk 4:
--------------------
2021 2007     2001    2015          2004 테러는 우리나라와 상관없지 않나요 최근 국내에서 테러단체에 동조하는 혐의로 적발되는 사례가 꾸준히 증가하고 있으며 해외에서는 우리 국민 대상으로 혐오 테러가 지속적으로 발생하고 있습니다. 드론을 활용하거나 차량 돌진 우편물이나 택배를 이용하는 등 새로운 유형의 테러도 우리 주변에서 언제든 발생할 수 있습니다. 테러는 국적도 인종도 지역도 가리지 않습니다. 테러에 안전지대는 없습니다. 세계를 뒤흔든 테러 사건 테러대비 행동요령 (www.nctc.go.kr)  0706  테러대비 행동요령 (www.nctc.go.kr)


Chunk 5:
--------------------
/5 /5 /5   

## 임베딩

In [33]:
# 1. OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 로컬 파일 저장소 설정 (사용자 환경에 맞는 경로로 설정)
store = LocalFileStore("F:/STUDY/sparta/999/박성규/emb")  # 로컬 경로 설정 각자 작성 해야합닏.

# 3. 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 모델 이름을 네임스페이스로 설정
)


'''
# 4. 예시 텍스트
example_texts = [
    "OpenAI의 임베딩 모델은 강력한 자연어 처리를 제공합니다.",
    "로컬 캐시는 효율적인 데이터 처리를 가능하게 합니다.",
    "임베딩 모델을 활용하여 고품질의 애플리케이션을 개발할 수 있습니다."
]

# 5. 임베딩 생성 및 저장된 결과 출력
for text in example_texts:
    embedding = cached_embedder.embed_query(text)  # 임베딩 생성
    print(f"텍스트: {text}")
    print(f"임베딩 (첫 10개 값): {embedding[:10]}")  # 임베딩 값의 일부를 출력

print("임베딩 생성 완료 및 로컬 캐시에 저장되었습니다.")
'''

'\n# 4. 예시 텍스트\nexample_texts = [\n    "OpenAI의 임베딩 모델은 강력한 자연어 처리를 제공합니다.",\n    "로컬 캐시는 효율적인 데이터 처리를 가능하게 합니다.",\n    "임베딩 모델을 활용하여 고품질의 애플리케이션을 개발할 수 있습니다."\n]\n\n# 5. 임베딩 생성 및 저장된 결과 출력\nfor text in example_texts:\n    embedding = cached_embedder.embed_query(text)  # 임베딩 생성\n    print(f"텍스트: {text}")\n    print(f"임베딩 (첫 10개 값): {embedding[:10]}")  # 임베딩 값의 일부를 출력\n\nprint("임베딩 생성 완료 및 로컬 캐시에 저장되었습니다.")\n'

In [34]:
vectorstore = FAISS.from_documents(documents=splits, embedding=cached_embedder)

In [35]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4}) 

# 테스트 하면서 수정할 예정

## Fuctiong calling 함수 정의

In [37]:
# 대피소 정보 불러오기
with open("SourceCode/shelters.json", "r", encoding="utf-8") as f:
    shelters = json.load(f)

In [39]:
# shelters 리스트를 데이터프레임으로 변환
df = pd.DataFrame(shelters)
df = df[df['SHNT_PSBLTY_NOPE'] != '0']  # 대피가능인원수 0인 곳을 제외

In [41]:
print(df.head())

  GRND_UDGD_SE ORTM_UTLZ_TYPE   SGG_CD     FCLT_CD    FCLT_SE_CD  \
0            1            주차장  3910000  S202400002  3              
1            1            주차장  3910000  S202400001  3              
2            1            주차장  4020000  S202400001  3              
3            1          지하주차장  3650000  S201700010  3              
4            1          지하주차장  3650000  S201700011  3              

     ROAD_NM_CD  LOT_MIN SHNT_PSBLTY_NOPE  FCLT_SCL  LOT_SEC  ... LAT_PROVIN  \
0  3352297             6            34761     28678       24  ...         37   
1  3352295             6            37152     30651       27  ...         37   
2  3182030            56            39997     32998       42  ...         37   
3  3165028            23             6577      5426       55  ...         36   
4  3165028            23             5126      4229       56  ...         36   

  LAT_MIN MNG_INST_TELNO                     FCLT_NM  FCLT_DSGN_DAY  \
0       1  031-8024-4900  평택지제역동문굿모닝힐맘시

In [42]:
# DMS를 소수점 좌표로 변환하는 함수 (경도 위도 각각 숫자 합치기)
def dms_to_dd(degrees, minutes, seconds):
    return degrees + minutes / 60 + seconds / 3600

# 위도 변환 (LAT_PROVIN, LAT_MIN, LAT_SEC를 사용)
df['위도'] = df.apply(lambda row: dms_to_dd(
    float(row['LAT_PROVIN']), 
    float(row['LAT_MIN']), 
    float(row['LAT_SEC'])
), axis=1)

# 경도 변환 (LOT_PROVIN, LOT_MIN, LOT_SEC를 사용)
df['경도'] = df.apply(lambda row: dms_to_dd(
    float(row['LOT_PROVIN']), 
    float(row['LOT_MIN']), 
    float(row['LOT_SEC'])
), axis=1)

# 주소 컬럼 설정 (도로명 주소 사용)
df['주소'] = df['FCLT_ADDR_RONA']
# 장소 컬럼 설정
df['시설명'] = df['FCLT_NM']

# 필요한 컬럼만 선택 추가가 가능하다. (대피소 정보 불러오고 추가 정보가 필요하다고 판단되면 추가하면 됩니다.) 답변 테스트 할때, 필요하다고 판단되면
df_result = df[['시설명','주소', '위도', '경도']]

print(df_result.head())

                          시설명                                        주소  \
0  평택지제역동문굿모닝힐맘시티4단지 지하주차장 1층  경기도 평택시 신촌5로 56 (칠원동, 평택지제역동문굿모닝힐맘시티4단지)   
1  평택지제역동문굿모닝힐맘시티2단지 지하주차장 1층  경기도 평택시 신촌3로 12 (칠원동, 평택지제역동문굿모닝힐맘시티2단지)   
2             힐스테이트 금정역 지하주차장         경기도 군포시 엘에스로 143 (금정동, 힐스테이트 금정역)   
3                유등마을아파트 104동           대전광역시 중구 수침로 138 (태평동, 유등마을아파트)   
4                유등마을아파트 106동           대전광역시 중구 수침로 138 (태평동, 유등마을아파트)   

          위도          경도  
0  37.025278  127.106667  
1  37.026389  127.107500  
2  37.373333  126.945000  
3  36.332222  127.398611  
4  36.331667  127.398889  


In [43]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    위도와 경도를 받아 두 지점 사이의 거리를 킬로미터 단위로 계산하는 함수
    """
    # 위도와 경도를 라디안으로 변환
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)

    # 위도와 경도의 차이 계산
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine 공식 적용
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371  # 지구 반지름 (킬로미터 단위)
    distance = R * c
    return distance

## 카카오맵 API 호출

In [45]:
''' 키워드 검색시 주소를 호출하지 못함 -> 키워드 검색 로직 추가 '''

load_dotenv()
kakaoapikey = os.getenv("REST_API_KEY") # 카카오 API 호출
def get_coordinates(query):
    """
    검색 질의어(query)를 기반으로 주소 검색 API와 장소 검색 API를 순차적으로 호출하여 경도와 위도를 반환.

    Args:
        query (str): 검색할 주소 또는 장소.
        kakaoapikey (str): 카카오맵 REST API 키.

    Returns:
        tuple: (longitude, latitude) - 경도(x), 위도(y)
    """
    # 1. 주소 검색 API 호출
    address_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakaoapikey}"}
    params = {"query": query, "size": 1}

    address_response = requests.get(address_url, headers=headers, params=params)
    if address_response.status_code == 200:
        data = address_response.json()
        documents = data.get("documents", [])
        if documents:
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
    
    # 2. 장소 검색 API 호출 (주소 검색 결과가 없을 경우)
    keyword_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    keyword_response = requests.get(keyword_url, headers=headers, params=params)
    if keyword_response.status_code == 200:
        data = keyword_response.json()
        documents = data.get("documents", [])
        if documents:
            x = documents[0].get("x")
            y = documents[0].get("y")
            return (x, y)
    
    # 결과 없음
    print("검색 결과가 없습니다.")
    return None


# 테스트 실행
if __name__ == "__main__":
    query = "동대구역"  # 검색할 장소 또는 주소
    coordinates = get_coordinates(query)
    if coordinates:
        print(f"경도: {coordinates[0]}, 위도: {coordinates[1]}")

경도: 128.628393775388, 위도: 35.8793239931795


In [46]:
def find_nearest_shelters(address: str) -> str:
    """
    주어진 주소를 기반으로 가까운 대피소 n개 정보를 반환하는 함수.
    """
    coordinates = get_coordinates(address)
    if coordinates:
        user_lon = float(coordinates[0])
        user_lat = float(coordinates[1])

        
        def calculate_distance(row):
            try:
                shelter_lat = float(row['위도'])
                shelter_lon = float(row['경도'])
                return haversine_distance(user_lat, user_lon, shelter_lat, shelter_lon)
            except (ValueError, TypeError):
                return None

        # 거리 계산 및 데이터프레임 업데이트
        df_result['거리'] = df_result.apply(calculate_distance, axis=1)
        df_valid = df_result[df_result['거리'].notnull()]
        df_sorted = df_valid.sort_values(by='거리')
        df_top3 = df_sorted.head(3)

        # 결과 문자열 생성
        result = "\n가장 가까운 대피소 정보 (거리순):"
        # 추후에 대피소 정보를 가져오는 테스트할때, 코드 변경할 수 있습니다.
        for idx, row in df_top3.iterrows():
            result += f"\n\n[{idx+1}]"
            result += f"\n시설명: {row['시설명']}"
            result += f"\n주소: {row['주소']}"
            result += f"\n현재 위치로부터의 거리: {row['거리']:.2f} km"

            shelter_add = row['주소'].replace(' ', '') 
            kakao_map_link = f"https://map.kakao.com/link/search/{shelter_add}"
            result += f"\n지도 링크: {kakao_map_link}"
        return result
    else:
        return "좌표를 가져올 수 없습니다."


In [47]:
# 함수 스키마 정의
functions = [
    {
        "name": "find_nearest_shelters",
        "description": "주어진 주소를 기반으로 가장 가까운 대피소 정보를 반환하는 함수.",
        "parameters": {
            "type": "object",
            "properties": {
                "address": {
                    "type": "string",
                    "description": "대피소를 찾고자 하는 주소"
                }
            },
            "required": ["address"]
        }
    }
]

## Streamlit 에서 위치기반 서비스 API를 이용하여, 사용자 위치를 불러와서 대피소 정보를 반환하는 것으로 추가기능 구현 예정임

In [50]:
# 모델 정의
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    model_kwargs={"functions": functions}
)